<a target="_blank" href="https://colab.research.google.com/github/IngCarlaPezzone/tensorflow-1-public/blob/main/C1/W4/ungraded_labs/C1_W4_Lab_3_compacted_images_traducido.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# # Instale este paquete para utilizar la GPU de Colab para el entrenamiento
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

In [ ]:
# Training data
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

In [ ]:
# Validation data
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

In [ ]:
import zipfile
# Unzip
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')

local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

zip_ref.close()

In [ ]:
import os

train_horse_dir = os.path.join('./horse-or-human/horses')
train_human_dir = os.path.join('./horse-or-human/humans')
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
print(f'TRAIN SET HORSES: {train_horse_names[:10]}')

train_human_names = os.listdir(train_human_dir)
print(f'TRAIN SET HUMANS: {train_human_names[:10]}')

validation_horse_hames = os.listdir(validation_horse_dir)
print(f'VAL SET HORSES: {validation_horse_hames[:10]}')

validation_human_names = os.listdir(validation_human_dir)
print(f'VAL SET HUMANS: {validation_human_names[:10]}')

In [ ]:
print(f'total training horse images: {len(os.listdir(train_horse_dir))}')
print(f'total training human images: {len(os.listdir(train_human_dir))}')
print(f'total validation horse images: {len(os.listdir(validation_horse_dir))}')
print(f'total validation human images: {len(os.listdir(validation_human_dir))}')

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    # 0 == 'horse'. 1 == 'human'
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#  Normalization
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flujo de imágenes de entrenamiento en lotes de 128 utilizando el generador train_datagen
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  #  Este es el directorio de origen para las imágenes de entrenamiento
        target_size=(150, 150),  # Todas las imágenes serán redimensionadas a 150x150
        batch_size=128,
        # Como se ha utilizado la pérdida binaria_crossentropy, se necesitan etiquetas binarias
        class_mode='binary')

# Flujo de imágenes de entrenamiento en lotes de 128 utilizando el generador train_datagen
validation_generator = validation_datagen.flow_from_directory(
        './validation-horse-or-human/',  # Este es el directorio de origen para las imágenes de entrenamiento
        target_size=(150, 150),  # Todas las imágenes se redimensionarán a 150x150
        batch_size=32,
        # Como se ha utilizado la pérdida binaria_crossentropy, se necesitan etiquetas binarias
        class_mode='binary')

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
    # predicting images
    path = '/content/' + fn
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    if classes[0]>0.5:
      print(fn + " is a human")
    else:
      print(fn + " is a horse")
 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Definir un nuevo modelo que tomará una imagen como entrada, y emitirá
# representaciones intermedias para todas las capas del modelo anterior después de
# la primera.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Prepara una imagen de entrada aleatoria del conjunto de entrenamiento.
horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)
img = load_img(img_path, target_size=(150, 150))  # esta es una imagen PIL
x = img_to_array(img)  # matriz Numpy con forma (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Matriz Numpy con forma (1, 150, 150, 3)

# Escala por  1/255
x /= 255

# Pasar la imagen por la red, obteniendo así todas
# representaciones intermedias de esta imagen.
successive_feature_maps = visualization_model.predict(x)

# Estos son los nombres de las capas, por lo que puede tenerlos como parte de la trama
layer_names = [layer.name for layer in model.layers[1:]]

# Visualizar las representaciones
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    if len(feature_map.shape) == 4:

    # Sólo se hace esto para las capas conv / maxpool, no para las capas totalmente conectadas
    n_features = feature_map.shape[-1]  # number of features in feature map

    # El mapa de características tiene forma (1, size, size, n_features)
    size = feature_map.shape[1]

    # Mosaico de las imágenes en esta matriz
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
        x = feature_map[0, :, :, i]
        x -= x.mean()
        x /= x.std()
        x *= 64
        x += 128
        x = np.clip(x, 0, 255).astype('uint8')

        # Colocar cada filtro en esta gran cuadrícula horizontal
        display_grid[:, i * size : (i + 1) * size] = x

    # Mostrar la grilla
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
# Terminate the kernel and free memory resources
import os, signal
os.kill(os.getpid(), signal.SIGKILL)